In [21]:
!python -V

Python 3.11.9


In [22]:
import pandas as pd

In [23]:
import pickle

In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [26]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/baran/project-courses/mlops-bootcamp/Week2/mlruns/1', creation_time=1718897934683, experiment_id='1', last_update_time=1718897934683, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [27]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

In [28]:
df_train = read_dataframe("data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("data/green_tripdata_2021-02.parquet")

In [29]:
len(df_train), len(df_val)

(73908, 61921)

In [30]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [31]:
categorical = ["PU_DO"]  #'PULocationID', 'DOLocationID']
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [32]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [33]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.75871521021275

In [34]:
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [35]:
with mlflow.start_run():

    mlflow.set_tag("developer", "baran_dev")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [36]:
import xgboost as xgb

In [37]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [38]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [39]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=20,
            evals=[(valid, "validation")],
            early_stopping_rounds=2,
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [40]:
search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 2, 50, 1)),
    "learning_rate": hp.loguniform("learning_rate", -2, 0),
    "reg_alpha": hp.loguniform("reg_alpha", -3, -1),
    "reg_lambda": hp.loguniform("reg_lambda", -2, -1),
    "min_child_weight": hp.loguniform("min_child_weight", -1, 2),
    "objective": "reg:linear",
    "seed": 42,
}

best_result = fmin(
    fn=objective, space=search_space, algo=tpe.suggest, max_evals=50, trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:41:29] WARNING: /croot/xgboost-split_1713972711803/work/cpp_src/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.41978                          
[1]	validation-rmse:9.17336                           
[2]	validation-rmse:8.32491                           
[3]	validation-rmse:7.76090                           
[4]	validation-rmse:7.38923                           
[5]	validation-rmse:7.14283                           
[6]	validation-rmse:6.98287                           
[7]	validation-rmse:6.87263                           
[8]	validation-rmse:6.79917                           
[9]	validation-rmse:6.74519                           
[10]	validation-rmse:6.71056                          
[11]	validation-rmse:6.68598                          
[12]	validation-rmse:6.66643                          
[13]	validation-rmse:6.65091                          
[14]	validation-rmse:6.63902                          
[15]	validation-rmse:6.63110                          
[16]	validation-rmse:6.62469                          
[17]	validation-rmse:6.61943                          
[18]	valid

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:41:35] WARNING: /croot/xgboost-split_1713972711803/work/cpp_src/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[4]	validation-rmse:6.82963                                                    
[5]	validation-rmse:6.80444                                                    
[6]	validation-rmse:6.79069                                                    
[7]	validation-rmse:6.78180                                                    
[8]	validation-rmse:6.77518                                                    
[9]	validation-rmse:6.76926                                                    
[10]	validation-rmse:6.76807                                                   
[11]	validation-rmse:6.76676                                                   
[12]	validation-rmse:6.76119                                                   
[13]	validation-rmse:6.75925                                                   
[14]	validation-rmse:6.75627                                                   
[15]	validation-rmse:6.75246                                                   
[16]	validation-rmse:6.74791            

...

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:44:43] WARNING: /croot/xgboost-split_1713972711803/work/cpp_src/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.59364                                                     
[1]	validation-rmse:6.75221                                                     
[2]	validation-rmse:6.59138                                                     
[3]	validation-rmse:6.53930                                                     
[4]	validation-rmse:6.52349                                                     
[5]	validation-rmse:6.51188                                                     
[6]	validation-rmse:6.50178                                                     
[7]	validation-rmse:6.49283                                                     
[8]	validation-rmse:6.48796                                                     
[9]	validation-rmse:6.48259                                                     
[10]	validation-rmse:6.47645                                                    
[11]	validation-rmse:6.47047                                                    
[12]	validation-rmse:6.46390

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:44:46] WARNING: /croot/xgboost-split_1713972711803/work/cpp_src/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.53098                                                     
[1]	validation-rmse:7.22834                                                     
[2]	validation-rmse:6.80220                                                     
[3]	validation-rmse:6.64960                                                     
[4]	validation-rmse:6.58657                                                     
[5]	validation-rmse:6.55333                                                     
[6]	validation-rmse:6.53408                                                     
[7]	validation-rmse:6.52260                                                     
[8]	validation-rmse:6.51744                                                     
[9]	validation-rmse:6.51332                                                     
[10]	validation-rmse:6.50660                                                    
[11]	validation-rmse:6.50257                                                    
[12]	validation-rmse:6.49809

/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [42]:
params = {
    "learning_rate": 0.7067657495012771,
    "max_depth": 49,
    "min_child_weight": 1.7041177164039096,
    "reg_alpha": 0.2802367356526795,
    "reg_lambda": 0.2024140613950744,
    "seed": 42,
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=20,
    evals=[(valid, "validation")],
    early_stopping_rounds=5,
)

2024/06/20 18:48:31 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c83a390c5d5b467cabef2d2cff07eee8', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:7.41881
[1]	validation-rmse:6.68866
[2]	validation-rmse:6.55331
[3]	validation-rmse:6.51388
[4]	validation-rmse:6.51093
[5]	validation-rmse:6.49951
[6]	validation-rmse:6.48918
[7]	validation-rmse:6.48246
[8]	validation-rmse:6.47719
[9]	validation-rmse:6.46704
[10]	validation-rmse:6.46078
[11]	validation-rmse:6.45064
[12]	validation-rmse:6.44622
[13]	validation-rmse:6.44225
[14]	validation-rmse:6.43907
[15]	validation-rmse:6.43326
[16]	validation-rmse:6.43121
[17]	validation-rmse:6.42468
[18]	validation-rmse:6.42087
[19]	validation-rmse:6.41753


2024/06/20 18:48:53 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/20 18:48:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:48:53] WARNING: /croot/xgboost-split_1713972711803/work/cpp_src/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/06/20 18:48:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/baran/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. Thi